In [38]:
import ray
import time
import os
import cv2 as cv
import skimage
import selective_search

start=time.perf_counter()

@ray.remote
def selective(image, src_path: str , dst_path: str):
  os.chdir(src_path)
  img = skimage.io.imread(image)

  boxes = selective_search.selective_search(img, mode='single')
  #boxes_filter = selective_search.box_filter(boxes, min_size=20, topN=80)

  # drawing rectangles on the original image
  img = cv.imread(image)
  for x1, y1, x2, y2 in boxes:
      img = cv.rectangle(img, (x1, y1), (x2, y2), color=(0,255,0), thickness=1)

  os.chdir(dst_path)
  cv.imwrite(image + "(selective).png", img)

if not ray.is_initialized():
    ray.init()

dataset="/content/resized"
dataset_list=os.listdir(dataset)
length=len(dataset_list)
threads=[]

for i in dataset_list:
  futures=selective.remote(i, dataset, "/content/selective")
  threads.append(futures)


for thread in threads:
  ray.get(thread)

ray.shutdown()

finish=time.perf_counter()

print("Total time: {}".format(round(finish-start,2)))
print("Total files in 'resized': {}".format(length))
print("Total files in 'selective': {}".format(len(os.listdir("/content/selective"))-1))


2023-07-01 12:04:48,464	INFO worker.py:1636 -- Started a local Ray instance.


Total time: 183.13
Total files in 'resized': 47
Total files in 'selective': 47


In [39]:

start=time.perf_counter()
def selective(image, src_path: str , dst_path: str):
  os.chdir(src_path)
  img = skimage.io.imread(image)

  boxes = selective_search.selective_search(img, mode='single')
  #boxes_filter = selective_search.box_filter(boxes, min_size=20, topN=80)

  # drawing rectangles on the original image
  img = cv.imread(image)
  for x1, y1, x2, y2 in boxes:
      img = cv.rectangle(img, (x1, y1), (x2, y2), color=(0,255,0), thickness=1)

  os.chdir(dst_path)
  cv.imwrite(image + "(selective).png", img)

dataset="/content/resized"
dataset_list=os.listdir(dataset)
for i in dataset_list:
  selective(i, dataset, "/content/selective2")

finish=time.perf_counter()
print("Total time: {}".format(round(finish-start,2)))
print("Total files in 'resized': {}".format(length))
print("Total files in 'selective': {}".format(len(os.listdir("/content/selective"))-1))


Total time: 175.33
Total files in 'resized': 47
Total files in 'selective': 47
